Matlab 5dB: /home/thien/Code/H_predict_UDA/H_predict_Sionna/Generate_Data/CDL_Channel/generatedChannel/ver3_/5dB/mapBaseData.mat
Sionna 5dB: /home/thien/Code/H_predict_UDA/H_predict_Sionna/Generate_Data/Sionna/generatedChannel/ver3_/sionnaTrue.mat

# Code to calculate PAD of original datasets

(1) construct the data set U using both source and target representations of the training samples; 

(2) randomly split U in two subsets of equal size; 

(3) train linear SVMs on the first subset of U using a large range of C values; 

(4) compute the error of all obtained classifiers on the second subset of U; 

(5) use the lowest error to compute the PAD value

In [2]:
import os
notebook_dir = os.getcwd()
print(notebook_dir)

/home/thien/Code/H_predict_UDA/H_predict_Sionna/Domain_Adaptation/Domain_Adversarial


In [2]:
source_data_file_path = os.path.abspath(os.path.join(notebook_dir, '..', '..', 'Generate_Data', 'CDL_Channel', 
                                                    'generatedChannel', 'ver3_', '5dB', 'mapBaseData.mat'))
target_data_file_path = os.path.abspath(os.path.join(notebook_dir, '..', '..', 'Generate_Data', 'Sionna', 
                                                    'generatedChannel', 'ver3_'))
print(source_data_file_path)
print(target_data_file_path)

/home/thien/Code/H_predict_UDA/H_predict_Sionna/Generate_Data/CDL_Channel/generatedChannel/ver3_/5dB/mapBaseData.mat
/home/thien/Code/H_predict_UDA/H_predict_Sionna/Generate_Data/Sionna/generatedChannel/ver3_


In [3]:
import h5py

batch_size=16

# ============ Source data ==============
source_file = h5py.File(source_data_file_path, 'r')
H_true_source = source_file['H_true']
N_samp_source = H_true_source.shape[0]
print('N_samp_source = ', N_samp_source)

# ============ Target data ==============
target_file = h5py.File(target_data_file_path+'/sionnaTrue.mat', 'r')
H_true_target = target_file['H_true']
N_samp_target = H_true_target.shape[0]
print('N_samp_target = ', N_samp_target)

N_samp_source =  2048
N_samp_target =  2048


In [4]:
for key in source_file.keys():
        data = source_file[key]
        if isinstance(data, h5py.Dataset):
            print(f"{key}: shape={data.shape}, dtype={data.dtype}")

H_duplicate_2to5: shape=(2048, 792, 84), dtype=[('real', '<f8'), ('imag', '<f8')]
H_equalized_1_6_11: shape=(2048, 792, 3), dtype=[('real', '<f8'), ('imag', '<f8')]
H_linear_1_6_11: shape=(2048, 792, 3), dtype=[('real', '<f8'), ('imag', '<f8')]
H_linear_1to6: shape=(2048, 792, 84), dtype=[('real', '<f8'), ('imag', '<f8')]
H_linear_2to5: shape=(2048, 792, 84), dtype=[('real', '<f8'), ('imag', '<f8')]
H_time: shape=(2048, 168944), dtype=[('real', '<f8'), ('imag', '<f8')]
H_true: shape=(2048, 792, 154), dtype=[('real', '<f8'), ('imag', '<f8')]


In [5]:
import numpy as np
indices_source = np.arange(N_samp_source)
np.random.shuffle(indices_source)
indices_target = np.arange(N_samp_target)
np.random.shuffle(indices_target)
#
train_size = int(np.floor(N_samp_source * 0.9) // batch_size * batch_size)
val_size = N_samp_source - train_size
print('train_size = ', train_size)
print('val_size = ', val_size)

# Repeat the indices to match the maximum number of samples
N_samp = max(N_samp_source, N_samp_target) 
indices_source = np.resize(indices_source, N_samp)
indices_target = np.resize(indices_target, N_samp)

# =======================================================
## Divide the indices into training and validation sets
indices_train_source = indices_source[:train_size]

indices_train_target = indices_target[:train_size]


train_size =  1840
val_size =  208


In [6]:
import sys
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', '..')))
import Est_btween_CSIRS.helper.utils as utils_CNN
# =========== Source dataset ==============
loader_H_1_6_11_train_source = utils_CNN.H5BatchLoader(source_file, 'H_linear_1_6_11', batch_size=batch_size, shuffled_indices=indices_train_source)
    # channel at symbol 2 of slots 1,6,11 (channel corresponding to CSI-RS 1, 2, 3)

# =========== Target dataset ==============
loader_H_1_6_11_train_target = utils_CNN.H5BatchLoader(target_file, 'H_linear_1_6', batch_size=batch_size, shuffled_indices=indices_train_target)
    # channel at symbol 2 of slots 1,6 (channel corresponding to CSI-RS 1, 2)


2025-06-26 10:48:00.655932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750949280.670526   40645 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750949280.674880   40645 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750949280.686340   40645 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750949280.686352   40645 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750949280.686354   40645 computation_placer.cc:177] computation placer alr

## Code for SVM
Can explode the memory with large dataset

In [7]:
loader_H_1_6_11_train_source.reset()
loader_H_1_6_11_train_target.reset()
all_features = []
all_labels = []
#
for batch_idx in range(loader_H_1_6_11_train_target.total_batches):
    # batch_target: shape (batch_size, 792, 2)
    batch_target = loader_H_1_6_11_train_target.next_batch()  # (batch_size, 792,2)
    real_target = batch_target['real']  # (batch_size, 792,2)
    imag_target = batch_target['imag']  # (batch_size, 792,2)
    real_flat = real_target.reshape(real_target.shape[0], -1)  # (batch_size, 1584)
    imag_flat = imag_target.reshape(imag_target.shape[0], -1)  # (batch_size, 1584)
    combined_target = np.concatenate([real_flat, imag_flat], axis=1)  # (batch_size, 3168)
    target_labels = (np.ones(combined_target.shape[0], dtype=int))
    # 
    batch_source = loader_H_1_6_11_train_source.next_batch()  # (batch_size, 792, 3)
    batch_source = batch_source[:,:,0:2]
    real_source = batch_source['real']  # (batch_size, 792,2)
    imag_source = batch_source['imag']  # (batch_size, 792,2)
    real_flat_source = real_source.reshape(real_source.shape[0], -1)  # (batch_size, 1584)
    imag_flat_source = imag_source.reshape(imag_source.shape[0], -1)  # (batch_size, 1584)
    combined_source = np.concatenate([real_flat_source, imag_flat_source], axis=1)  # (batch_size, 3168)
    source_labels = (np.zeros(combined_source.shape[0], dtype=int))

    # --- Combine and append ---
    all_features.append(combined_source)
    all_features.append(combined_target)
    all_labels.append(source_labels)
    all_labels.append(target_labels)

# Stack all batches into a single dataset
X = np.vstack(all_features)  # shape: (n_samples, 3168)
y = np.concatenate(all_labels)  # shape: (n_samples,)



In [8]:
from sklearn.model_selection import train_test_split

# X: (3680, 3168), y: (3680,)
X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5, random_state=42, shuffle=True)

print(X1.shape, y1.shape)  # (1840, 3168) (1840,)
print(X2.shape, y2.shape)  # (1840, 3168) (1840,)

(1840, 3168) (1840,)
(1840, 3168) (1840,)


In [9]:
# Now train SVM
from sklearn.svm import SVC
C_values = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0, 500.0, 1000.0]
best_epsilon = 1.0
best_C = None
for C in C_values:
    svm = SVC(C=C, probability=True)
    svm.fit(X1, y1)
    accuracy = svm.score(X2, y2)
    error_rate = 1 - accuracy
    print(f"C: {C}, Error rate: {error_rate:.4f}")
    if error_rate < best_epsilon:
        best_epsilon = error_rate
        best_C = C
print(f"Best C: {best_C}, Best error rate: {best_epsilon:.4f}")
pad = 2 * (1 - 2 * best_epsilon)
print(f"PAD = {pad:.4f}")

C: 0.01, Error rate: 0.0245
C: 0.1, Error rate: 0.0239
C: 0.5, Error rate: 0.0212
C: 1.0, Error rate: 0.0201
C: 2.0, Error rate: 0.0120
C: 5.0, Error rate: 0.0011
C: 10.0, Error rate: 0.0011
C: 50.0, Error rate: 0.0011
C: 100.0, Error rate: 0.0011
C: 500.0, Error rate: 0.0011
C: 1000.0, Error rate: 0.0011
Best C: 5.0, Best error rate: 0.0011
PAD = 1.9957


# Code for SGDClassifier
Can train the classifier batch by batch

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# Standardize features
scaler = StandardScaler()
X1_scaled = scaler.fit_transform(X1)
X2_scaled = scaler.transform(X2)

alpha_values = np.logspace(-6, 0, 10)
best_epsilon_sgd = 1.0
best_alpha = None
for alpha in alpha_values:
    sgd = SGDClassifier(loss='hinge', alpha=alpha, max_iter=5000, tol=1e-5, random_state=42)
    sgd.fit(X1_scaled, y1)
    y2_pred = sgd.predict(X2_scaled)
    error_rate = 1 - accuracy_score(y2, y2_pred)
    print(f"alpha: {alpha}, Error rate: {error_rate:.4f}")
    if error_rate < best_epsilon_sgd:
        best_epsilon_sgd = error_rate
        best_alpha = alpha
print(f"Best alpha: {best_alpha}, Best error rate: {best_epsilon_sgd:.4f}")
pad_sgd = 2 * (1 - 2 * best_epsilon_sgd)
print(f"PAD (SGDClassifier) = {pad_sgd:.4f}")

alpha: 1e-06, Error rate: 0.0011
alpha: 4.641588833612782e-06, Error rate: 0.0011
alpha: 2.1544346900318823e-05, Error rate: 0.0011
alpha: 0.0001, Error rate: 0.0005
alpha: 0.00046415888336127773, Error rate: 0.0016
alpha: 0.002154434690031882, Error rate: 0.0011
alpha: 0.01, Error rate: 0.0016
alpha: 0.046415888336127725, Error rate: 0.0022
alpha: 0.21544346900318823, Error rate: 0.0033
alpha: 1.0, Error rate: 0.0201
Best alpha: 0.0001, Best error rate: 0.0005
PAD (SGDClassifier) = 1.9978


# Test SGD for batches
Loading from features_source.h5 and features_target.h5

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import h5py
import numpy as np

with h5py.File('features_source.h5', 'r') as f_source, h5py.File('features_target.h5', 'r') as f_target:
    n_source = f_source['features'].shape[0]
    n_target = f_target['features'].shape[0]
    total = n_source + n_target

    # Create index and label arrays
    indices = np.arange(total)
    labels = np.zeros(total, dtype=int)
    labels[n_source:] = 1  # target samples get label 1

    # # Shuffle indices
    np.random.seed(42)
    np.random.shuffle(indices)

    # Select half for your split
    split = total // 2
    train_indices = indices[:split]
    train_labels = labels[train_indices]
    test_indices = indices[split:]
    test_labels = labels[test_indices]

    batch_size = 16
    alpha_values = np.logspace(-6, 0, 10)
    best_epsilon_sgd = 1.0
    best_alpha = None
    
    # --- Train SGDClassifier batch by batch ---
    for alpha in alpha_values:
        clf = SGDClassifier(loss='hinge', alpha=alpha, max_iter=5000, tol=1e-5, random_state=42)
        classes = np.array([0, 1])
        first_batch = True
        

        for i in range(0, len(train_indices), batch_size):
            batch_indices = train_indices[i:i+batch_size]
            batch_labels = train_labels[i:i+batch_size]
            batch_features = []
            for idx in batch_indices:
                if idx < n_source:
                    feat = f_source['features'][idx]
                else:
                    feat = f_target['features'][idx - n_source]
                batch_features.append(feat)
            batch_features = np.stack(batch_features, axis=0)
            batch_features = batch_features.reshape(batch_features.shape[0], -1)
            if first_batch:
                clf.partial_fit(batch_features, batch_labels, classes=classes)
                first_batch = False
            else:
                clf.partial_fit(batch_features, batch_labels)

        # --- Test SGDClassifier batch by batch ---
        y_true = []
        y_pred = []
        for i in range(0, len(test_indices), batch_size):
            batch_indices = test_indices[i:i+batch_size]
            batch_labels = test_labels[i:i+batch_size]
            batch_features = []
            for idx in batch_indices:
                if idx < n_source:
                    feat = f_source['features'][idx]
                else:
                    feat = f_target['features'][idx - n_source]
                batch_features.append(feat)
            batch_features = np.stack(batch_features, axis=0)
            batch_features = batch_features.reshape(batch_features.shape[0], -1)
            preds = clf.predict(batch_features)
            y_true.extend(batch_labels)
            y_pred.extend(preds)

        error_rate = 1 - accuracy_score(y_true, y_pred)
        print(f"alpha: {alpha}, Error rate: {error_rate:.4f}")
        if error_rate < best_epsilon_sgd:
            best_epsilon_sgd = error_rate
            best_alpha = alpha

        
    pad = 2 * (1 - 2 * best_epsilon_sgd)
    print(f"PAD (SGDClassifier) = {pad:.4f}")

alpha: 1e-06, Error rate: 0.0729
alpha: 4.641588833612782e-06, Error rate: 0.0729
alpha: 2.1544346900318823e-05, Error rate: 0.0729
alpha: 0.0001, Error rate: 0.0729
alpha: 0.00046415888336127773, Error rate: 0.0729
alpha: 0.002154434690031882, Error rate: 0.0729
alpha: 0.01, Error rate: 0.0729
alpha: 0.046415888336127725, Error rate: 0.0729
alpha: 0.21544346900318823, Error rate: 0.0729
alpha: 1.0, Error rate: 0.0729
PAD (SGDClassifier) = 1.7083


In [1]:
from helper import PAD
pad = PAD.cal_PAD_SGD('features_source.h5', 'features_target.h5')

alpha: 1e-06, Error rate: 0.0729
alpha: 4.641588833612782e-06, Error rate: 0.0729
alpha: 2.1544346900318823e-05, Error rate: 0.0729
alpha: 0.0001, Error rate: 0.0729
alpha: 0.00046415888336127773, Error rate: 0.0729
alpha: 0.002154434690031882, Error rate: 0.0729
alpha: 0.01, Error rate: 0.0729
alpha: 0.046415888336127725, Error rate: 0.0729
alpha: 0.21544346900318823, Error rate: 0.0729
alpha: 1.0, Error rate: 0.0729
PAD (SGDClassifier) = 1.7083
